# Music Generator
## COMP 432 Fall 2020
## Natalia Whiteley 40044353 and Michael Naccache 27...

In [41]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from music21 import *

In [134]:
# instruments to include from the midi file
INSTRUMENTS = [instrument.Piano, instrument.Bass, instrument.Guitar, instrument.Saxophone, instrument.Trumpet]

## Part 1: Import Dataset and Split

In [141]:
# given the name of a midi file, returns a 2D array of notes per instrument
def import_midi(midi_file):    
    
    # parse midi file
    midi = converter.parse(midi_file)
  
    # seperate based on instrument
    midi_parts = instrument.partitionByInstrument(midi)

    notes = {}
    notes_to_parse = []

    for part in midi_parts.parts:
        # only use instruments we want
        if any(isinstance(part.getInstrument(), x) for x in INSTRUMENTS):
            notes[str(part.getInstrument().bestName())] = []
            notes_to_parse = part.recurse()

            # finding whether a particular element is note or a chord
            for element in notes_to_parse: 
                #note
                if isinstance(element, note.Note):
                    notes[str(part.getInstrument().bestName())].append(str(element.pitch))
                
                #chord
                elif isinstance(element, chord.Chord):
                    notes[str(part.getInstrument().bestName())].append('.'.join(str(n) for n in element.normalOrder))

    # remove any empty
    notes = {k: v for k, v in notes.items() if v}

    return notes

In [142]:
print(import_midi('a_night_in_tunisia_2_jc.mid'))

{'Acoustic Bass': ['E-2', 'B-2', 'C#3', 'E-3', 'C#3', 'B-2', 'D2', 'F2', 'A2', 'B2', 'A2', 'D2', 'E-2', 'B-2', 'C#3', 'E-3', 'C#3', 'B-2', 'D2', 'F2', 'A2', 'B2', 'A2', 'D2', 'E-2', 'B-2', 'C#3', 'E-3', 'C#3', 'B-2', 'D2', 'F2', 'A2', 'B2', 'A2', 'E2', 'D3', 'E2', 'A2', 'D2', 'C#3', 'B2', 'E-2', 'B-2', 'C#3', 'E-3', 'C#3', 'B-2', 'D2', 'F2', 'A2', 'B2', 'A2', 'D2', 'E-2', 'B-2', 'C#3', 'E-3', 'C#3', 'B-2', 'D2', 'F2', 'A2', 'B2', 'A2', 'D2', 'E-2', 'B-2', 'C#3', 'E-3', 'C#3', 'B-2', 'D2', 'F2', 'A2', 'B2', 'A2', 'E2', 'D3', 'E2', 'A2', 'D2', 'C#3', 'B2', 'E-2', 'B-2', 'E-3', 'B-2', 'E-2', 'D2', 'D1', 'E2', 'F2', 'A2', 'D2', 'E-2', 'E-1', 'C#2', 'C2', 'B-1', 'E-2', 'D2', 'E2', 'F2', 'A2', 'D2', 'E-2', 'E-1', 'C#2', 'C2', 'B-1', 'E-2', 'D2', 'E2', 'F2', 'A2', 'G#2', 'G2', 'G1', 'B-1', 'A1', 'A0', 'A1', 'D2', 'D1', 'E2', 'F2', 'A2', 'D2', 'E-2', 'C#2', 'C2', 'B-1', 'E-2', 'D2', 'E2', 'F2', 'A2', 'D2', 'E-2', 'F2', 'F#2', 'G2', 'E-2', 'D2', 'E2', 'F2', 'A2', 'E-2', 'F2', 'F#2', 'G2', 'E-2'

## Part 2: Data Analysis

In [ ]:
# graph distribution of all images

# graph distribution of train images

# graph distribution of test images

## Part 3: Normalize Dataset

## Part 3: Train GAN With Magenta/Wavenet

## Part 4: Train TensorFlow Models

## Part 5: Compare Results

## Sources

https://www.analyticsvidhya.com/blog/2020/01/how-to-perform-automatic-music-generation/